In [33]:
import os, sys
import numpy as np
import time
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path
import yaml

cwd = os.getcwd()
dirLib = cwd + r'/../utils'
if dirLib not in sys.path:
    sys.path.append(dirLib)

from generate_sim_data import gen_rho, gen_sim_data

In [39]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        config = yaml.safe_load(file)
    return config

In [48]:
config_path = cwd + r'\..\config'
config = load_config(config_path + r'\config_sims.yaml')

In [60]:
tD = float(config['params']['tD'])  # [s]
Nshot = config['params']['Nshot']
wrap_deadtime = config['params']['wrap_deadtime']
window_start = float(config['params']['window_start'])  # [s]
window_end = float(config['params']['window_end'])  # [s]
laser_pulse_width = float(config['params']['laser_pulse_width'])  # [s]
target_time = float(config['params']['target_time'])  # [s]
background = float(config['params']['background'])  # [Hz]
amplitude = np.array([float(config['params']['amplitude'])])  # [Hz]
t_sim_min = config['params']['t_sim_min']  # [s]
t_sim_max = float(config['params']['t_sim_max'])  # [s]
dt_sim = float(config['params']['dt_sim'])  # [s]

window_bnd = np.array([window_start, window_end])  # [s]

In [61]:
home = str(Path.home())
# save_dir = home + r'\OneDrive - UCB-O365\ARSENL\Experiments\SPCM\Data\Simulated\manuscript_revise_distributed'
save_dir = home + r'\OneDrive - UCB-O365\ARSENL\Experiments\SPCM\Data\Simulated\none'

In [62]:
# To iterate at different widths and scales
scale = 100
target_time *= scale  # [s]
laser_pulse_width *= scale  # [s] laser pulse width (sigma) 
window_bnd *= scale  # [s] time-of-flight bounds
t_sim_max *= scale  # [s]
dt_sim *= scale  # [s]

In [63]:
for i in range(len(amplitude)):
    
    target_amplitude = amplitude[i]
    
    ### GENERATE SIMULATED DATA ###
    det_idx, phot_idx, sync_idx, det_sync_idx, phot_sync_idx, det_events, phot_events = gen_sim_data(t_sim_max, dt_sim,
                                                                                                     tD, Nshot,
                                                                                                     wrap_deadtime,
                                                                                                     window_bnd,
                                                                                                     laser_pulse_width,
                                                                                                     target_time,
                                                                                                     target_amplitude,
                                                                                                     background)

    time_tag_index = det_idx
    true_time_tag_index = phot_idx
    sync_index = sync_idx
    time_tag = det_events
    true_time_tag = phot_events
    time_tag_sync_index = det_sync_idx
    true_time_tag_sync_index = phot_sync_idx

    # Save simulated data to netCDF
    sim_data = xr.Dataset(
        data_vars=dict(
            time_tag=(['time_tag_index'], time_tag),
            time_tag_sync_index=(['time_tag_index'], time_tag_sync_index),
            true_time_tag=(['true_time_tag_index'], true_time_tag),
            true_time_tag_sync_index=(['true_time_tag_index'], true_time_tag_sync_index),
            laser_pulse_width=laser_pulse_width,
            target_time=target_time,
            target_amplitude=target_amplitude,
            background=background,
            window_bnd=window_bnd,
            dt_sim=dt_sim
        ),
        coords=dict(
            sync_index=(['sync_index'], sync_index)
        )
    )

    fname = r'\sim_amp{:.1E}_nshot{:.1E}_width{:.1E}_dt{:.1E}.nc'.format(target_amplitude, Nshot, laser_pulse_width, dt_sim)
    sim_data.to_netcdf(save_dir+fname)
    
print('(First 10) Time tags:', time_tag[:10])

time elapsed: 0.00899958610534668


PermissionError: [Errno 13] Permission denied: b'C:\\Users\\Grant\\OneDrive - UCB-O365\\ARSENL\\Experiments\\SPCM\\Data\\Simulated\\none\\sim_amp1.0E+04_nshot1.0E+00_width5.0E-08_dt2.5E-09.nc'